In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "distilbert/distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
